In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
print(f'number of columns is {df.shape[1]}')

number of columns is 19


In [3]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [4]:
duration_std = np.std(df['duration'])
print(f'standard deviation of the trips duration in January is {duration_std}')

standard deviation of the trips duration in January is 42.5943442974141


In [5]:
df_ = df[(df.duration >= 1) & (df.duration <= 60)]
percentage_left = (df_.shape[0]/df.shape[0])*100
print(f'fraction of the records left after dropping the outliers is {percentage_left}')

fraction of the records left after dropping the outliers is 98.1220282212598


In [6]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [7]:
train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(3009173, 516)

In [8]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)**0.5

7.649140464167203

In [9]:
df2 = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df2['duration'] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime
df2.duration = df2.duration.apply(lambda td: td.total_seconds() / 60)
df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)]

In [10]:
df2[categorical] = df2[categorical].astype(str)
val_dicts = df2[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df2[target].values
y_pred2 = lr.predict(X_val)
mean_squared_error(y_val, y_pred2)**0.5

7.811462911207304